# Notebook for downloading/refreshing trainign data

## 1. Download data from Bloomberg for training
Note: 
- in the future the download script will be rewritten to adapt to a REST api
of our choice.
- Use the `asia_dd_env` environment for downloading data from bbg.

In [1]:
import pandas as pd
from xbbg import blp
import numpy as np
from typing import Union, Optional, Any, List, Dict

In [ ]:
# market cap > 500m
universe = pd.read_csv("./datafiles/mpax_universe_all.csv", header=[0], index_col=[0], low_memory=False)
universe = universe.query("`CUR_MKT_CAP_USD`>1e3")
figi_list = universe.ID_BB_GLOBAL

In [ ]:
training_desc = universe.filter(["SECURITY_NAME", "PARSEKYABLE_DES_SOURCE", 
                "GICS_SUB_INDUSTRY", "ID_ISIN", "UD_ECONOMIC_CORRELATION", 
                "UD_ALGO_RATING", "ID_BB_GLOBAL"])

In [ ]:
training_desc

,SECURITY_NAME,PARSEKYABLE_DES_SOURCE,GICS_SUB_INDUSTRY,ID_ISIN,UD_ECONOMIC_CORRELATION,UD_ALGO_RATING,ID_BB_GLOBAL
1,ITOCHU Corp,8001 JP Equity,20107010,JP3143600009,Cyclical,2B,BBG000B9WJ55
5,Enerpac Tool Group Corp,ATU US Equity,20106020,US2927651040,Cyclical,3,BBG000B9WX45
8,Tatneft PJSC,ATAD LI Equity,10102020,US8766292051,Commodity,7+,BBG000B9X7K3
9,Ameren Corp,AEE US Equity,55103010,US0236081024,Defensive,2A,BBG000B9X8C0
10,Woodside Petroleum Ltd,WPL AU Equity,10102020,AU000000WPL2,Commodity,3+,BBG000B9XBS6
...,...,...,...,...,...,...,...
22510,Apogee Therapeutics Inc,APGE US Equity,35201010,US03770N1019,Defensive,NaN,BBG01H51WYQ5
22511,BGC Group Inc,BGC US Equity,40203020,US0889291045,Cyclical,NaN,BBG01H9FTGX5
22512,Atlanta Braves Holdings Inc,BATRA US Equity,50202010,US0477261046,Cyclical,NaN,BBG01HCDRG86
22513,Atlanta Braves Holdings Inc,BATRK US Equity,50202010,US0477263026,Cyclical,NaN,BBG01HCX3Y34


In [ ]:
bbg_fields = dict(
    return_fields = {"return_com_eqy", "normalized_roe", "operating_roic", "return_on_asset"},
    margin_fields = {"ebitda_margin", "gross_margin", "ebit_margin", "eff_tax_rate",
            "fcf_margin_after_oper_lea_pymt"},
    is_fields = {"sales_rev_turn", "net_income", "is_rd_expend", 
                 "ardr_selling_general_admin_exp", 
            "is_selling_expenses", "is_opex_adjusted", "tot_int_exp"
            "cf_cap_expend_prpty_add", "cf_cash_from_oper"},
    leverage_fields = {"total_debt_to_tot_eqy", "net_debt_to_shrhldr_eqty", 
            "net_debt_to_ebitda", "fixed_charge_coverage_ratio"},
    bs_ratios = {"invent_days", "acct_rcv_days", "days_accounts_payable", 
            "cash_conversion_cycle", },
    est_fields = {"best_sales", "best_gross_margin", "best_net_income"},
    best_overrides = [{"best_fperiod_override": "1FY"}, {"best_fperiod_override": "2FY"}, {"best_fperiod_override": "3FY"}]
    )

In [ ]:
import itertools
default_columns = pd.MultiIndex.from_tuples(
    (itertools.chain(*[[(k, i) for i in v] for k, v in bbg_fields.items() 
                       if k not in ("est_fields", "default_override", "best_overrides")])))

In [ ]:
from typing import Collection, Sequence, Literal, Dict
import datetime as dt


TODAY = dt.datetime.today().date()

def get_hist_financials(
        tickers: Collection[str], 
        start_date: dt.date=dt.date(1995, 1, 1), 
        end_date: dt.date=TODAY):
    hist_fields = set()
    for fld_name, fld in bbg_fields.items():
        if fld_name not in ("est_fields", "default_override", "best_overrides"):
            hist_fields = hist_fields.union(fld)
    hist_financials = blp.bdh(tickers, hist_fields, start_date=start_date, 
                    end_date=end_date, 
                    Per="Y",
                    # **bbg_fields.get("default_override")
                    )
    hist_financials.index = hist_financials.index.astype("datetime64[ns]").to_series().apply(lambda d: pd.Period(d, freq="Y"))
    hist_financials = hist_financials.rename_axis("year", axis=0).rename_axis(["figi", "field"], axis=1)
    hist_financials = hist_financials.reset_index().groupby("year").mean().stack()\
        .unstack(0).reindex(default_columns.get_level_values(1))
    hist_financials.index = default_columns[default_columns.get_level_values(1).isin(hist_financials.index)]

    # calculated ratios fields
    temp_ratios = hist_financials.loc['is_fields'].drop(['sales_rev_turn'], axis=0) / hist_financials.loc['is_fields'].loc['sales_rev_turn']
    temp_ratios.index = pd.MultiIndex.from_product((["margins"], temp_ratios.index.to_series().apply(lambda x: f"{x}_to_sales").values))
    
    # calculated growth fields
    temp_growth = hist_financials.loc['is_fields'].loc[["sales_rev_turn", "net_income"]]
    temp_growth = temp_growth.stack(1).unstack(0).pct_change(periods=1).stack(1).unstack(0)
    temp_growth.index = pd.MultiIndex.from_product((["growth"], temp_growth.index.to_series().apply(lambda x: f"{x}_growth").values))

    res = pd.concat([hist_financials.drop("is_fields", axis=0), temp_growth])
    return res

# def get_estimates(
#         tickers,
#         start_date: dt.date=dt.date(1995, 1, 1), 
#         end_date: dt.date=TODAY):


def get_price_multiples(
        tickers: Collection[str], 
        start_date: dt.date=dt.date(2000, 1, 1), 
        end_date: dt.date=TODAY):
    df = blp.bdh(tickers, ["px_last", "best_cur_ev_to_ebitda", 
                "fcf_yield_with_cur_entp_val", "best_pe_next_ear",
                "px_to_book_ratio", "px_to_sales_ratio"],
                start_date=start_date,
                end_date=end_date,
                Per="W", 
                )
    df.index = df.index.astype("datetime64[ns]").to_period(freq="W")
    df = df.ffill()
    return df
    
def get_future_returns(ref_date: dt.date, price_df: pd.DataFrame):
    df = price_df.copy(deep=True)
    df.index = df.index.astype("datetime64[ns]")
    base_date_ind = df.index.to_series().lt(np.datetime64(ref_date)).sum()
    _3m_ind = df.index.to_series().lt(np.datetime64(ref_date + dt.timedelta(days=90))).sum()
    _6m_ind = df.index.to_series().lt(np.datetime64(ref_date + dt.timedelta(days=180))).sum()
    _1yr_ind = df.index.to_series().lt(np.datetime64(ref_date + dt.timedelta(days=365))).sum()
    _3yr_ind = df.index.to_series().lt(np.datetime64(ref_date + dt.timedelta(days=365 * 3))).sum()
    base_price = df.iloc[base_date_ind]
    _3m_returns= (df.iloc[ _3m_ind - 5 : _3m_ind + 5] / base_price).mean()
    # _3m_returns_std = (df.iloc[ _3m_ind - 5 : _3m_ind + 5] / base_price).std()
    _6m_returns = (df.iloc[ _6m_ind - 10 : _6m_ind + 10] / base_price).mean()
    _1yr_returns = (df.iloc[ _1yr_ind - 20 : _1yr_ind + 20] / base_price).mean()
    _3yr_returns = (df.iloc[ _3yr_ind - 60 : _3yr_ind + 60] / base_price).mean()
    res = pd.concat([_3m_returns, _6m_returns, _1yr_returns, _3yr_returns], axis=1)
    res.columns = ["3m", "6m", "1yr", "3yr"]
    return res


In [ ]:
hist_financial_data = get_hist_financials(training_desc.ID_BB_GLOBAL)
hist_financial_data.to_csv("historical_financial_data.csv")

C:\Users\p.peng\AppData\Local\Temp\ipykernel_17916\3596706963.py:19: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  hist_financials = hist_financials.reset_index().groupby("year").mean().stack()\
C:\Users\p.peng\AppData\Local\Temp\ipykernel_17916\3596706963.py:32: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  res = pd.concat([hist_financials.drop("is_fields", axis=0), temp_growth])


In [ ]:
px = get_price_multiples(training_desc.ID_BB_GLOBAL)

In [ ]:
px.to_csv("price_multiples.csv")

## 2. Organizing data for SQL server

In [2]:
# universe = pd.read_csv("./datafiles/impax_universe_all.csv", 
#                        header=[0], index_col=[0], low_memory=False)
# financial_data = pd.read_csv("./datafiles/historical_financial_data.csv", 
#                              header=[0,1], index_col=[0,1])
# financial_data_forsql = financial_data.T
# financial_data_forsql.columns = financial_data_forsql.columns.get_level_values(1)
# financial_data_forsql.reset_index().to_csv(
#     "./datafiles/historical_financial_data_sql.csv",
#     index=False)
price_multiples = pd.read_csv("./datafiles/price_multiples.csv", 
                             header=[0,1], index_col=[0])

In [3]:
import os, sys
os.chdir("../")
sys.path.append('c:\\Users\\p.peng\\StockEncoder')
from utils.database import SQLDatabase

In [4]:
import datetime as dt
price_multiples.index.names = ["period"]
price_multiples.columns.names = ["figi", "field"]
price_multiples_pivot = price_multiples\
    .unstack().to_frame("value")\
        .pivot_table(index=["figi", "period"], columns=["field"], aggfunc="mean")  # remove duplicates by averaging
price_multiples_pivot.columns = price_multiples_pivot.columns.get_level_values(1)
price_multiples_pivot = price_multiples_pivot.reset_index()
price_multiples_pivot["period"] = price_multiples_pivot.period.apply(
    lambda x: dt.datetime.strptime(x.split("/")[-1], "%Y-%m-%d")).values

In [5]:
price_multiples_pivot = price_multiples_pivot\
            .replace(float("inf"), np.nan)\
            .replace(float("-inf"), np.nan)

In [12]:
# trouble shooting

chunk_size = 10000
for i in tqdm(range(900000, len(price_multiples_pivot), chunk_size)):
    print(i)
    price_multiples_pivot\
                .iloc[i: i + chunk_size].to_sql( 
                "price_multiples_stock_encoder", 
                index=False, 
                if_exists="append",
                con=sql_engine
                )

# price_multiples_pivot\
#             .replace(float("inf"), np.nan)\
#             .replace(float("-inf"), np.nan)\
#             .iloc[(i+1) * chunk_size - 1 : (i+1) * chunk_size].to_sql( 
#             "price_multiples_stock_encoder", 
#             index=False, 
#             if_exists="append",
#             con=sql_engine
#             )

 86%|████████▌ | 606/703 [4:08:27<34:06, 21.10s/it]

6960000


 86%|████████▋ | 607/703 [4:08:51<35:02, 21.90s/it]

6970000


 86%|████████▋ | 608/703 [4:09:24<39:59, 25.25s/it]

...

100%|█████████▉| 701/703 [4:48:32<00:51, 25.60s/it]

7910000


100%|█████████▉| 702/703 [4:48:53<00:24, 24.32s/it]

7920000


100%|██████████| 703/703 [4:49:01<00:00, 24.67s/it]


In [6]:
from tqdm import tqdm
import time
import threading
import json


class TimeoutException(Exception):
    pass

def timeout(seconds):
    def decorator(func):
        def wrapper(*args, **kwargs):
            res = [TimeoutException('function timeout')]
            def target(result, *args, **kwargs):
                try:
                    result[0] = func(*args, **kwargs)
                except Exception as e:
                    result[0] = e
            thread = threading.Thread(target=target, args=(res,)+args, kwargs=kwargs)
            thread.start()
            thread.join(seconds)
            if isinstance(res[0], BaseException):
                raise res[0]
            return res[0]
        return wrapper

def timeout_no_return_func(seconds):
    """timeout decorator for functions that returns None"""
    def decorator(func):
        def wrapper(*args, **kwargs):
            res = [TimeoutException('function timeout')]
            def target(result, *args, **kwargs):
                try:
                    func(*args, **kwargs)
                    result[0] = None  # Explicitly set result to None if function doesn't return anything
                except Exception as e:
                    print(e)
                    result[0] = e
            thread = threading.Thread(target=target, args=(res,)+args, kwargs=kwargs)
            thread.start()
            thread.join(seconds)
            if isinstance(res[0], BaseException):
                raise res[0]
            return res[0]
        return wrapper
    return decorator

sql = SQLDatabase()
sql_engine = sql.engine
chunk_size = 50
last_i = 101
if f"passed_price_multiple_index_chunksize={chunk_size}.json" in os.listdir("./data"):
    passed_i = json.load(open(f"./data/passed_price_multiple_index_chunksize={chunk_size}.json", "r"))
else:
    passed_i = []
# price_multiples_pivot\
#     .replace(float("inf"), np.nan)\
#     .replace(float("-inf"), np.nan)\
#     .iloc[: chunk_size]\
#     .to_sql(
#     "price_multiples_stock_encoder", 
#     index=False, 
#     if_exists="replace", # 1st write, need to replace
#     con=sql_engine
#     )

# @timeout_no_return_func(15)
def load_chunk_to_sql(i: int):
    if (i + 1) * chunk_size <= len(price_multiples) - 1:
        price_multiples_pivot\
            .iloc[(i)*chunk_size : (i+1) * chunk_size]\
            .to_sql( 
            "price_multiples_stock_encoder", 
            index=False, 
            if_exists="append",
            con=sql_engine
            )
    else:
        price_multiples_pivot\
            .iloc[(i)*chunk_size : ]\
            .to_sql(
            "price_multiples_stock_encoder", 
            index=False, 
            if_exists="append",
            con=sql_engine
            )
    return True

for i in tqdm(range(last_i, len(price_multiples_pivot) // chunk_size)):
    try:
        load_chunk_to_sql(i)
    except TimeoutException:
        pass
    passed_i.append(i)
    json.dump(passed_i, open(f"./data/passed_price_multiple_index_chunksize={chunk_size}.json", "w"))
    last_i = i 

  0%|          | 0/158323 [02:12<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
universe = pd.read_csv("impax_universe_all.csv", header=[0], index_col=[0], low_memory=False)

In [ ]:
financial_data = pd.read_csv("historical_financial_data.csv", header=0, index_col=0)

## 3. Load data into database